In [ ]:
import numpy as np
import os
import time
import itertools
from math import ceil

class Benchmark:
    def __init__(self, file_path, benchmark_type):
        self.file_path = file_path
        self.benchmark_type = benchmark_type
        self.subsets, self.universe_size, self.num_subsets = self.read_benchmark()

    def read_benchmark(self):
        """Lecture d'un benchmark MCP à partir d'un fichier texte"""
        with open(self.file_path, "r") as file:
            lines = file.readlines()

        m, n = map(int, lines[0].split())
        cost_lines_to_skip = ceil(n / 12) if self.benchmark_type == "4" else ceil(n / 15)
        subset_start_index = 1 + cost_lines_to_skip  # Start of subset coverage data
        
        data_lines = lines[subset_start_index:]
        row_to_subsets = {}
        index = 0

        for row in range(1, m + 1):
            num_subsets = int(data_lines[index].strip())
            index += 1

            subsets = []
            while len(subsets) < num_subsets:
                subsets.extend(map(int, data_lines[index].split()))
                index += 1

            row_to_subsets[row] = subsets

        subset_to_rows = {}
        for row, subsets in row_to_subsets.items():
            for subset in subsets:
                if subset not in subset_to_rows:
                    subset_to_rows[subset] = []
                subset_to_rows[subset].append(row)

        return subset_to_rows, m, n

class PSO_MCP:
    def __init__(self, benchmark, num_particles=100, num_iterations=50, c1=1.5, c2=2, w=1, vmax=10, early_stop=30):
        self.benchmark = benchmark
        self.num_particles = num_particles
        self.num_iterations = num_iterations
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.vmax = vmax
        self.early_stop = early_stop
        self.k = ceil(benchmark.universe_size * 2 / 3)  # Define k

    def fitness_func(self, solution):
        """Évalue une solution en mesurant le nombre d'éléments couverts"""
        covered_elements = set()
        for idx, selected in enumerate(solution):
            if selected and idx in self.benchmark.subsets:
                covered_elements.update(self.benchmark.subsets[idx])
        return len(covered_elements), len(covered_elements) / self.benchmark.universe_size

    def run(self):
        """Exécute l'optimisation PSO"""
        np.random.seed(42)
        particles = np.zeros((self.num_particles, self.benchmark.universe_size), dtype=int)
        
        for i in range(self.num_particles):
            ones_indices = np.random.choice(self.benchmark.universe_size, self.k, replace=False)
            particles[i, ones_indices] = 1
        
        velocities = np.zeros((self.num_particles, self.benchmark.universe_size))
        personal_best = particles.copy()
        personal_best_scores = np.array([self.fitness_func(p)[1] for p in particles])
        global_best_idx = np.argmax(personal_best_scores)
        global_best = personal_best[global_best_idx].copy()
        global_best_covered, global_best_score = self.fitness_func(global_best)
        
        best_scores = []
        no_improvement = 0
        start_time = time.time()

        for iteration in range(self.num_iterations):
            for i in range(self.num_particles):
                velocities[i] = self.update_velocity_binary(velocities[i], particles[i], personal_best[i], global_best)
                particles[i] = self.binary_update(particles[i], velocities[i])
                
                num_covered, score = self.fitness_func(particles[i])
                
                if score > personal_best_scores[i]:
                    personal_best[i] = particles[i].copy()
                    personal_best_scores[i] = score
                
                if score > global_best_score:
                    global_best = particles[i].copy()
                    global_best_covered, global_best_score = num_covered, score
                    no_improvement = 0  # Reset early stop counter

            best_scores.append(global_best_score)
            print(f"Iteration {iteration+1}/{self.num_iterations} - Covered: {global_best_covered}, Score: {global_best_score * 100:.4f} %")

            if no_improvement >= self.early_stop:
                print(f"Arrêt anticipé après {iteration+1} itérations.")
                break
            
            no_improvement += 1

        execution_time = time.time() - start_time
        print(f"Optimisation terminée en {execution_time:.2f} sec - Meilleure couverture: {global_best_covered}, Score final: {global_best_score *100 :.4f} %")
        
        return global_best, global_best_covered, global_best_score, execution_time

        # # Tracer l'évolution du score
        # plt.plot(best_scores, label='Best Score')
        # plt.xlabel('Iterations')
        # plt.ylabel('Score')
        # plt.title('Évolution du score pendant l'optimisation')
        # plt.legend()
        # plt.show()

    def update_velocity_binary(self, velocity, particle, personal_best, global_best):
        """Binary PSO velocity update using discrete difference"""
        r1, r2 = np.random.rand(len(particle)), np.random.rand(len(particle))
        velocity = (self.w * velocity +
                    self.c1 * r1 * (personal_best != particle).astype(int) +
                    self.c2 * r2 * (global_best != particle).astype(int))
        return np.clip(velocity, -4, 4)

    def enforce_k_constraint(self, particle):
        """S'assure que le vecteur binaire a exactement k éléments activés"""
        if np.sum(particle) != self.k:
            ones = np.where(particle == 1)[0]
            zeros = np.where(particle == 0)[0]

            if len(ones) > self.k:
                np.random.shuffle(ones)
                particle[ones[self.k:]] = 0  # Désactive les surplus
            elif len(ones) < self.k:
                np.random.shuffle(zeros)
                particle[zeros[:(self.k - len(ones))]] = 1  # Active des zéros pour atteindre k
        return particle
    

    def binary_update(self, particle, velocity):
        """Mise à jour des particules en utilisant la transformation sigmoïde"""
        prob = 1 / (1 + np.exp(-velocity))
        updated_particle = (np.random.rand(*particle.shape) < prob).astype(int)
        return self.enforce_k_constraint(updated_particle)


    def update_velocity_binary(self, velocity, particle, personal_best, global_best):
        """Binary PSO velocity update using discrete difference"""
        r1, r2 = np.random.rand(len(particle)), np.random.rand(len(particle))
        velocity = (self.w * velocity +
                    self.c1 * r1 * (personal_best != particle).astype(int) +
                    self.c2 * r2 * (global_best != particle).astype(int))
        return np.clip(velocity, -self.vmax, self.vmax)

def hyperparameter_tuning(benchmark):
    """Teste plusieurs configurations de PSO et trouve les meilleurs paramètres"""
    param_grid = {
        'num_iterations': [50, 100, 200], 
        'w': [0.4, 0.6, 0.8],  
        'c1': [1, 1.5, 2.0, 2.5],  
        'c2': [1.0, 1.5, 2.0, 2.5],  
        'vmax': [6, 8, 10, 15, 20],  
        'num_particles': [50, 75, 100]  
    }
    
    best_score = 0
    best_params = None
    all_combinations = list(itertools.product(*param_grid.values()))
    print(f" {len(all_combinations)} configurations à tester...\n")

    for i, (num_iterations, w, c1, c2, vmax, num_particles) in enumerate(all_combinations):
        print(f" Test {i+1}/{len(all_combinations)} - Iterations: {num_iterations}, W: {w}, c1: {c1}, c2: {c2}, Vmax: {vmax}, Particles: {num_particles}")

        pso = PSO_MCP(benchmark, num_particles, num_iterations, c1, c2, w, vmax)
        _, _, score, _ = pso.run()
        
        if score > best_score:
            best_score = score
            best_params = (num_iterations, w, c1, c2, vmax, num_particles)

    print(" Meilleure configuration trouvée :")
    print(f"    Iterations: {best_params[0]}, W: {best_params[1]}, c1: {best_params[2]}, c2: {best_params[3]}, Vmax: {best_params[4]}, Particles: {best_params[5]}")
    print(f"    Meilleur Score: {best_score:.4f}")
    return best_params

if __name__ == "__main__":
    benchmark = Benchmark("Benchmark/C/scpc3.txt", "C")
    hyperparameter_tuning(benchmark)


 1152 configurations à tester...

 Test 1/1152 - Iterations: 50, W: 0.4, c1: 1, c2: 1.0, Vmax: 6, Particles: 50
Iteration 1/50 - Covered: 399, Score: 99.7500 %
Iteration 2/50 - Covered: 399, Score: 99.7500 %
Iteration 3/50 - Covered: 399, Score: 99.7500 %
Iteration 4/50 - Covered: 399, Score: 99.7500 %
Iteration 5/50 - Covered: 399, Score: 99.7500 %
Iteration 6/50 - Covered: 399, Score: 99.7500 %
Iteration 7/50 - Covered: 399, Score: 99.7500 %
Iteration 8/50 - Covered: 399, Score: 99.7500 %
Iteration 9/50 - Covered: 399, Score: 99.7500 %
Iteration 10/50 - Covered: 399, Score: 99.7500 %
Iteration 11/50 - Covered: 399, Score: 99.7500 %
Iteration 12/50 - Covered: 399, Score: 99.7500 %
Iteration 13/50 - Covered: 399, Score: 99.7500 %
Iteration 14/50 - Covered: 399, Score: 99.7500 %
Iteration 15/50 - Covered: 399, Score: 99.7500 %
Iteration 16/50 - Covered: 399, Score: 99.7500 %
Iteration 17/50 - Covered: 399, Score: 99.7500 %
Iteration 18/50 - Covered: 399, Score: 99.7500 %
Iteration 19/50

4 ==>
Iterations: 50, W: 0.5, c1: 2.5, c2: 2.5, Vmax: 6, Particles: 100

#A ==>   
Iterations: 50, W: 0.4, c1: 1, c2: 1.0, Vmax: 6, Particles: 50

#B ==>
Iterations: 50, W: 0.5, c1: 1.5, c2: 1.0, Vmax: 10, Particles: 50

#C ==>
Iterations: 50, W: 0.4, c1: 1, c2: 1.0, Vmax: 6, Particles: 50
